<a href="https://colab.research.google.com/github/taylorec/Design-Patterns-with-Python/blob/main/3)_Structural_Design_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A structural design pattern proposes a way of composing objects to provide new functionality.

## The Adapter pattern

The adapter pattern is a structural design pattern that helps us make two incompatible interfaces compatible.

In [ ]:
class OldPaymentSystem:
    def __init__(self, currency):
        self.currency = currency
    def make_payment(self, amount):
        print(
            f"[OLD] Pay {amount} {self.currency}"
        )

In [ ]:
class NewPaymentGateway:
    def __init__(self, currency):
        self.currency = currency
    def execute_payment(self, amount):
        print(
            f"Execute payment of {amount} {self.currency}"
        )

In [ ]:
class PaymentAdapter:
    def __init__(self, system):
        self.system = system
    def make_payment(self, amount):
        self.system.execute_payment(amount)

In [ ]:
def main():
    old_system = OldPaymentSystem("euro")
    print(old_system)
    new_system = NewPaymentGateway("euro")
    print(new_system)
    adapter = PaymentAdapter(new_system)
    adapter.make_payment(100)

In [ ]:
if __name__ == "__main__":
    main()

Execute payment of 100 euro


## The Decorator pattern

The decorator pattern allows added responsibilities to an object dynamically, and in a transparent manner (without affecting other objects).

A Python decorator is a callable (function, method, or class) that gets a func_in function object as input and returns another function object, func_out. It is a commonly used technique for extending the behavior of a function, method, or class.

In [ ]:
import functools

In [ ]:
def memoize(func):
    cache = {}
    @functools.wraps(func)
    def memoizer(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return memoizer

In [ ]:
@memoize
def number_sum(n):
    if n == 0:
        return 0
    else:
        return n + number_sum(n - 1)

In [ ]:
@memoize
def fibonacci(n):
    if n in (0, 1):
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
def main():
    from timeit import Timer
    to_execute = [
        (
            number_sum,
            Timer(
                "number_sum(300)",
                "from __main__ import number_sum",
            ),
        ),
        (
            fibonacci,
            Timer(
                "fibonacci(100)",
                "from __main__ import fibonacci",
            ),
        ),
    ]
    for item in to_execute:
        func = item[0]
        print(
            f'Function "{func.__name__}": {func.__doc__}'
        )
        t = item[1]
        print(f"Time: {t.timeit()}")
        print()

In [ ]:
if __name__ == "__main__":
    main()

Function "number_sum": None
Time: 0.285784460999821

Function "fibonacci": None
Time: 0.2241251939999529



## The Bridge pattern

The bridge pattern  is designed up-front to decouple an implementation from its abstraction.

Using the bridge pattern is a good idea when you want to share an implementation among multiple objects. Basically, instead of implementing several specialized classes, and defining all that is required within each class, you can define the following special components:

- An abstraction that applies to all the classes
- A separate interface for the different objects involved

In [ ]:
import abc
import urllib.parse
import urllib.request

In [ ]:
class ResourceContent:
    """
    Define the abstraction's interface.
    Maintain a reference to an object which represents the Implementor.
    """

    def __init__(self, imp):
        self._imp = imp

    def show_content(self, path):
        self._imp.fetch(path)

In [ ]:
class ResourceContentFetcher(metaclass=abc.ABCMeta):
    """
    Define the interface (Implementor) for implementation classes that help fetch content.
    """

    @abc.abstractmethod
    def fetch(path):
        pass

In [ ]:
class URLFetcher(ResourceContentFetcher):
    """
    Implement the Implementor interface and define its concrete
    implementation.
    """

    def fetch(self, path):
        # path is an URL
        req = urllib.request.Request(path)
        with urllib.request.urlopen(req) as response:
            if response.code == 200:
                the_page = response.read()
                print(the_page)

In [ ]:
class LocalFileFetcher(ResourceContentFetcher):
    """
    Implement the Implementor interface and define its concrete
    implementation.
    """

    def fetch(self, path):
        # path is the filepath to a text file
        with open(path) as f:
            print(f.read())

In [ ]:
def main():
    url_fetcher = URLFetcher()
    iface = ResourceContent(url_fetcher)
    iface.show_content('http://python.org')

    print('===================')

    localfs_fetcher = LocalFileFetcher()
    iface = ResourceContent(localfs_fetcher)
    iface.show_content('file.txt')

In [ ]:
if __name__ == "__main__":
    main()

## The Facade pattern

The facade design pattern helps hide the internal complexity of systems and expose only what is necessary to the client through a simplified interface. In essence, facade is an abstraction layer implemented over an existing complex system.

In [ ]:
from enum import Enum
from abc import ABCMeta, abstractmethod

In [ ]:
State = Enum('State', 'new running sleeping restart zombie')

In [ ]:
class User:
    pass

In [ ]:
class Process:
    pass

In [ ]:
class File:
    pass

In [ ]:
class Server(metaclass=ABCMeta):
    @abstractmethod
    def __init__(self):
        pass

    def __str__(self):
        return self.name

    @abstractmethod
    def boot(self):
        pass

    @abstractmethod
    def kill(self, restart=True):
        pass

In [ ]:
class FileServer(Server):
    def __init__(self):
        '''actions required for initializing the file server'''
        self.name = 'FileServer'
        self.state = State.new

    def boot(self):
        print(f'booting the {self}')
        '''actions required for booting the file server'''
        self.state = State.running

    def kill(self, restart=True):
        print(f'Killing {self}')
        '''actions required for killing the file server'''
        self.state = State.restart if restart else State.zombie

    def create_file(self, user, name, permissions):
        '''check validity of permissions, user rights, etc.'''
        print(f"trying to create the file '{name}' for user '{user}' with permissions {permissions}")

In [ ]:
class ProcessServer(Server):
    def __init__(self):
        '''actions required for initializing the process server'''
        self.name = 'ProcessServer'
        self.state = State.new

    def boot(self):
        print(f'booting the {self}')
        '''actions required for booting the process server'''
        self.state = State.running

    def kill(self, restart=True):
        print(f'Killing {self}')
        '''actions required for killing the process server'''
        self.state = State.restart if restart else State.zombie

    def create_process(self, user, name):
        '''check user rights, generate PID, etc.'''
        print(f"trying to create the process '{name}' for user '{user}'")

In [ ]:
class WindowServer:
    pass

In [ ]:
class NetworkServer:
    pass

In [ ]:
class OperatingSystem:
    '''The Facade'''
    def __init__(self):
        self.fs = FileServer()
        self.ps = ProcessServer()

    def start(self):
        [i.boot() for i in (self.fs, self.ps)]

    def create_file(self, user, name, permissions):
        return self.fs.create_file(user, name, permissions)

    def create_process(self, user, name):
        return self.ps.create_process(user, name)

In [ ]:
def main():
    os = OperatingSystem()
    os.start()
    os.create_file('foo', 'hello', '-rw-r-r')
    os.create_process('bar', 'ls /tmp')

In [ ]:
if __name__ == '__main__':
    main()

booting the FileServer
booting the ProcessServer
trying to create the file 'hello' for user 'foo' with permissions -rw-r-r
trying to create the process 'ls /tmp' for user 'bar'


## The Flyweight pattern

The flyweight design pattern is a technique used to minimize memory usage and improve performance by introducing data sharing between similar objects. A flyweight is a shared object that contains state-independent, immutable (also known as intrinsic) data. The state-dependent, mutable (also known as extrinsic) data should not be part of flyweight because this is information that cannot be shared, since it differs per object. If flyweight needs extrinsic data, it should be provided explicitly by the client code.

In [ ]:
import random
from enum import Enum

In [ ]:
CarType = Enum('CarType', 'subcompact compact suv')

In [ ]:
class Car:
    pool = dict()

    def __new__(cls, car_type):
        obj = cls.pool.get(car_type, None)
        if not obj:
            obj = object.__new__(cls)
            cls.pool[car_type] = obj
            obj.car_type = car_type
        return obj

    def render(self, color, x, y):
        type = self.car_type
        msg = f'render a car of type {type} and color {color} at ({x}, {y})'
        print(msg)

In [ ]:
def main():
    rnd = random.Random()
    #age_min, age_max = 1, 30    # in years
    colors = 'white black silver gray red blue brown beige yellow green'.split()
    min_point, max_point = 0, 100
    car_counter = 0

    for _ in range(10):
        c1 = Car(CarType.subcompact)
        c1.render(random.choice(colors),
                  rnd.randint(min_point, max_point),
                  rnd.randint(min_point, max_point))
        car_counter += 1

    for _ in range(3):
        c2 = Car(CarType.compact)
        c2.render(random.choice(colors),
                  rnd.randint(min_point, max_point),
                  rnd.randint(min_point, max_point))
        car_counter += 1

    for _ in range(5):
        c3 = Car(CarType.suv)
        c3.render(random.choice(colors),
                  rnd.randint(min_point, max_point),
                  rnd.randint(min_point, max_point))
        car_counter += 1

    print(f'cars rendered: {car_counter}')
    print(f'cars actually created: {len(Car.pool)}')

    c4 = Car(CarType.subcompact)
    c5 = Car(CarType.subcompact)
    c6 = Car(CarType.suv)
    print(f'{id(c4)} == {id(c5)}? {id(c4) == id(c5)}')
    print(f'{id(c5)} == {id(c6)}? {id(c5) == id(c6)}')

In [ ]:
if __name__ == '__main__':
    main()

render a car of type CarType.subcompact and color red at (23, 1)
render a car of type CarType.subcompact and color white at (37, 61)
render a car of type CarType.subcompact and color yellow at (59, 51)
render a car of type CarType.subcompact and color gray at (96, 26)
render a car of type CarType.subcompact and color red at (59, 44)
render a car of type CarType.subcompact and color black at (72, 50)
render a car of type CarType.subcompact and color silver at (61, 97)
render a car of type CarType.subcompact and color green at (19, 66)
render a car of type CarType.subcompact and color black at (90, 92)
render a car of type CarType.subcompact and color black at (18, 72)
render a car of type CarType.compact and color white at (31, 97)
render a car of type CarType.compact and color yellow at (30, 19)
render a car of type CarType.compact and color silver at (35, 35)
render a car of type CarType.suv and color beige at (73, 21)
render a car of type CarType.suv and color white at (48, 87)
rende

## The Proxy pattern

Proxy is defined as having the authority to represent another object.

In [ ]:
class SensitiveInfo:
    def __init__(self):
        self.users = ['nick', 'tom', 'ben', 'mike']

    def read(self):
        nb = len(self.users)
        print(f"There are {nb} users: {' '.join(self.users)}")

    def add(self, user):
        self.users.append(user)
        print(f'Added user {user}')

In [ ]:
class Info:
    '''protection proxy to SensitiveInfo'''

    def __init__(self):
        self.protected = SensitiveInfo()
        self.secret = '0xdeadbeef'

    def read(self):
        self.protected.read()

    def add(self, user):
        sec = input('what is the secret? ')
        self.protected.add(user) if sec == self.secret else print("That's wrong!")

In [ ]:
def main():
    info = Info()

    while True:
        print('1. read list |==| 2. add user |==| 3. quit')
        key = input('choose option: ')
        if key == '1':
            info.read()
        elif key == '2':
            name = input('choose username: ')
            info.add(name)
        elif key == '3':
            return False
        else:
            print(f'unknown option: {key}')

In [ ]:
if __name__ == '__main__':
    main()

1. read list |==| 2. add user |==| 3. quit
choose option: 1
There are 4 users: nick tom ben mike
1. read list |==| 2. add user |==| 3. quit
choose option: 2
choose username: Spongebob
what is the secret? 0xdeadbeef
Added user Spongebob
1. read list |==| 2. add user |==| 3. quit
choose option: 1
There are 5 users: nick tom ben mike Spongebob
1. read list |==| 2. add user |==| 3. quit
choose option: 3
